# Week 3 Assignment - Part 1

Download dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!pip install beautifulsoup4
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


Scrape data from Wikipedia

In [2]:
# GET request
input_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [3]:
# Parse HTML to Beautiful Soup
soup = BeautifulSoup(input_data, "html.parser")
#soup

In [4]:
# Lists
postcode = []
borough = []
neighborhood = []

In [5]:
#Table

for i in soup.find("table").find_all("tr"):
    x = i.find_all("td")
    if (len(x) > 0):
        postcode.append(x[0].text.rstrip("\n"))
        borough.append(x[1].text.rstrip("\n"))
        neighborhood.append(x[2].text.rstrip("\n"))    

In [6]:
#Dataframe
df = pd.DataFrame({"PostalCode": postcode,
                  "Borough": borough,
                  "Neighborhood": neighborhood})
print(df.head())

  PostalCode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront


In [7]:
#drop not assigned
df_clean = df[df.Borough != "Not assigned"].reset_index(drop = True)

print(df_clean.head())

  PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


In [8]:
df_grouped = df_clean.groupby(["PostalCode", "Borough"], as_index = False).agg(lambda x: ", ".join(x))
print(df_grouped.head())

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae


In [9]:
for index, i in df_grouped.iterrows():
    if i["Neighborhood"] == "Not assigned":
        i["Neighborhood"] = i["Borough"]
        
print(df_grouped.head())

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae


In [10]:
#shape
df_grouped.shape

(103, 3)

# Week 3 Assignment - Part 2

In [11]:
#add geo data
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#rename
geo.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#merge
df_new = df_grouped.merge(geo, on="PostalCode", how="left")
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Week 3 Assignment - Part 3

In [14]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
#focus on scarborough

scarborough_data = df_new[df_new['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [18]:
CLIENT_ID = 'BH50HIWF5OEL21Z5X3HEI0FEZPNWXTM5FONWSAON1GDQO2C1' # your Foursquare ID
CLIENT_SECRET = 'P15LUDHOWEGL2RTGSMQLJZ2FKS2MFIVPXLBPWU2PLHBZM4FN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [19]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['PostalCode'], scarborough_data['Borough'], scarborough_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venues_df.head()
venues_df.shape

(93, 9)

In [21]:
venues_df.groupby(["Neighborhood"]).count()

,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
Agincourt,5,5,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",8,8,8,8,8,8,8,8
"Guildwood, Morningside, West Hill",9,9,9,9,9,9,9,9
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5,5,5


In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 55 uniques categories.


In [23]:
# one hot encoding
scarborough_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
scarborough_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(scarborough_onehot.columns[-1:]) + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()
scarborough_onehot.shape

(93, 56)

In [24]:
scarborough_grouped = scarborough_onehot.groupby("Neighborhoods").mean().reset_index()
scarborough_grouped.shape
scarborough_grouped

,Neighborhoods,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.000000,0.200000,0.000,0.00,0.000,0.000000,0.2,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.2,0.2,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.200000,0.00,0.000,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.25,0.000,0.000000,0.0,0.0000,0.25,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.25,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.250000,0.00,0.000,0.000000,0.0
2,Cedarbrae,0.000000,0.125,0.125,0.125000,0.000000,0.000000,0.000,0.00,0.125,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.125000,0.0000,0.125000,0.00,0.000,0.000000,0.125,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000,0.083333,0.000000,0.000000,0.000,0.00,0.000,0.083333,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.083333,0.083333,0.0000,0.083333,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.083333,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.083333,0.000000,0.0,0.166667,0.166667,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.083333,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.333333,0.00,0.000,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.200000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.4,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.250,0.000000,0.000000,0.000000,0.125,0.00,0.000,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.125,0.0,0.125000,0.000000,0.0,0.0,0.0,0.000000,0.125,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.125,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000,0.111111,0.000000,0.111111,0.000,0.00,0.000,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.111111,0.000000,0.000000,0.0000,0.000000,0.00,

In [25]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.12
1    Athletics & Sports  0.12
2           Gas Station  0.12
3      Hakka Restaurant  0.12
4   Fried Chicken Joint  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0              Pharmacy  0.17
1           Pizza Place  0.17
2    Italian Restaurant  0.08
3  Fast Food Restaurant  0.08
4          Noodle House  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1 

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = scarborough_grouped['Neighborhoods']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Construction & Landscaping,Gym,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Department Store,Gym
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Italian Restaurant,Chinese Restaurant,Thai Restaurant,Bank,Gas Station,Fried Chicken Joint,Noodle House,Fast Food Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Skating Rink,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [28]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop("Neighborhoods", 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 4, 0, 4, 4, 4, 3, 1], dtype=int32)

In [29]:
kmeans.labels_.shape

(16,)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latiude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Vietnamese Restaurant,Construction & Landscaping,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,Home Service,Construction & Landscaping,Bar,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Mexican Restaurant,Medical Center,Bank,Moving Target,Restaurant,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Soccer Field,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Department Store,Gym


In [31]:

# sort the results by Cluster Labels
print(scarborough_merged.shape)
scarborough_merged = scarborough_merged.drop([scarborough_merged.index[16]])
scarborough_merged.sort_values(["Cluster Labels"], inplace=True)
scarborough_merged

(17, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Motel,Skating Rink,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0.0,College Stadium,Skating Rink,General Entertainment,Café,Construction & Landscaping,Gym,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,0.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Gas Station
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Vietnamese Restaurant,Construction & Landscaping,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2.0,Smoke Shop,Playground,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Soccer Field,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,3.0,Convenience Store,Discount Store,Department Store,Coffee Shop,Hobby Shop,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,Home Service,Construction & Landscaping,Bar,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Mexican Restaurant,Medical Center,Bank,Moving Target,Restaurant,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Department Store,Gym


In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,0.0,American Restaurant,Motel,Skating Rink,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,Scarborough,0.0,College Stadium,Skating Rink,General Entertainment,Café,Construction & Landscaping,Gym,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint
12,Scarborough,0.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Gas Station


In [34]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Vietnamese Restaurant,Construction & Landscaping,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store


In [35]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2.0,Smoke Shop,Playground,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


In [36]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3.0,Coffee Shop,Soccer Field,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
6,Scarborough,3.0,Convenience Store,Discount Store,Department Store,Coffee Shop,Hobby Shop,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station


In [37]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Home Service,Construction & Landscaping,Bar,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,Scarborough,4.0,Mexican Restaurant,Medical Center,Bank,Moving Target,Restaurant,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Discount Store
4,Scarborough,4.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Department Store,Gym
7,Scarborough,4.0,Bakery,Bus Station,Ice Cream Shop,Park,Intersection,Metro Station,Soccer Field,Gas Station,Furniture / Home Store,Convenience Store
10,Scarborough,4.0,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Construction & Landscaping,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
11,Scarborough,4.0,Middle Eastern Restaurant,Smoke Shop,Bakery,Breakfast Spot,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
13,Scarborough,4.0,Pizza Place,Pharmacy,Italian Restaurant,Chinese Restaurant,Thai Restaurant,Bank,Gas Station,Fried Chicken Joint,Noodle House,Fast Food Restaurant
14,Scarborough,4.0,Gym,Playground,Park,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
15,Scarborough,4.0,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Gym,Pharmacy,Discount Store,Coffee Shop,Electronics Store,Pizza Place,Furniture / Home Store
